# WEAVE-Clusers LIFU workflow for SV

This is a small example, akin to `TUTORIAL.md`. Please read that file (and `README`) for more details.

This notebook is copied from "WA_example.ipynb" by Yago Ascasibar and modified for the WC-LIFU SV by SCT.

## Initialisation

To import the different steps, we must add the `ifu` directory (the parent of the `workflow` directory where this Notebook is supposed to live) to the Python path.

In [1]:
import os
import sys
import numpy as np
from astropy.coordinates import SkyCoord, FK5, ICRS
import logging
import glob

# create logger
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))
logger.setLevel(logging.INFO)
logger.handlers[0].stream = sys.stdout

if '..' not in sys.path:
    sys.path.append('..')
    print(sys.path)

['/Users/sctrager/Library/Mobile Documents/com~apple~CloudDocs/WEAVE/SWG/IFUworkflow/ifu/workflow', '/Users/sctrager/Python/anaconda3', '/Users/sctrager/Python/anaconda3/lib/python38.zip', '/Users/sctrager/Python/anaconda3/lib/python3.8', '/Users/sctrager/Python/anaconda3/lib/python3.8/lib-dynload', '', '/Users/sctrager/Python/anaconda3/lib/python3.8/site-packages', '/Users/sctrager/Python/anaconda3/lib/python3.8/site-packages/aeosa', '/Users/sctrager/Python/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/sctrager/.ipython', '..']


We also specify the path to the `configure` tools (`configure` and `dither`) needed in stage 4.

In [2]:
path_to_configure_tools = '../../../Configure/configure.app/Contents/MacOS/'

Set the location of the template files and the output directory (create the latter if necessary):

In [3]:
ifu_driver_template = os.path.join('aux', 'ifu_driver_template.fits')
fits_catalogue_template = os.path.join('aux', 'WC_CatalogueTemplate.fits')

output_dir = 'output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Stage 1: Creation of IFU driver catalogue (based on your input)

Fill the trimester data, as well as the e-mail addresses of the PI and SWG members:

In [4]:
survey = 'WC'
trimester = '2021A1'
PI_email = 'jalfonso.aguerri@iac.es'
PI_first_name = 'J. Alfonso'
PI_surname = 'Lopez Aguerri'
report_verbosity = 1
cc_report = 'sctrager@astro.rug.nl,jalberto@iac.es,daniela.bettoni@inaf.it,Ulrike.Kuchner@nottingham.ac.uk,jairo@iac.es'

For the WC LIFU SV observations, we will observe 3 OBs: 
- the center of Abell 2142 with a 1-hour OB with the standard 3-position dither
- an infalling group near Abell 2142 with a 1-hour OB with the standard 3-position dither
- the same infalling group near Abell 2142 with a 1-hour OB with the "-3"-position dither (to complete the full coverage of the group with the LIFU)

In [5]:
data_dict = {}

data_dict['TARGSRVY'] = 3*[survey]
data_dict['TARGPROG'] = 3*['WCC']

data_dict['TARGID'] = ['A2142center','A2142infall','A2142infall']
data_dict['TARGNAME'] = ['Abell 2142','Abell 2142A','Abell 2142A']

data_dict['TARGPRIO'] = [10.0, 10.0, 10.0]

data_dict['PROGTEMP'] = ['41332', '41332', '41332']

data_dict['OBSTEMP'] = ['NBDAB', 'NBDAB', 'NBDAB']

data_dict['GAIA_RA'] = [239.58333333, 239.71149583, 239.71149583]
data_dict['GAIA_DEC'] = [27.23341944, 27.39375833, 27.39375833]

data_dict['GAIA_PMRA'] = [np.nan, np.nan, np.nan]
data_dict['GAIA_PMDEC'] = [np.nan, np.nan, np.nan]
data_dict['GAIA_PARAL'] = [np.nan, np.nan, np.nan]

data_dict['IFU_PA_REQUEST'] = [0.0, 0.0, 0.0]

data_dict['IFU_DITHER'] = [3, 3, -3]

Convert to the Gaia reference frame, ICRS 2015.5 (see https://ingconfluence.ing.iac.es/confluence/display/WEAVEDEV/Precession+to+Gaia+reference+frame)

In [6]:
necessary = True  # coordinates are actually J2000
# necessary = False  # coordinates are already ICRS

if necessary:
    original_coords = SkyCoord(ra=data_dict['GAIA_RA'], dec=data_dict['GAIA_DEC'], unit='deg', frame=FK5(equinox='J2000'))
    #coords_intermediate_Gaia = original_coords.transform_to(FK5(equinox='J2015.5'))
    #coords_Gaia = coords_intermediate_Gaia.transform_to(ICRS())
    coords_Gaia = original_coords.transform_to(ICRS())
    data_dict['GAIA_RA'] = coords_Gaia.ra.value
    data_dict['GAIA_DEC'] = coords_Gaia.dec.value

#print(data_dict['GAIA_RA'])
#print(data_dict['GAIA_DEC'])
    
data_dict['GAIA_EPOCH'] = 3*['2015.5']

And that's it. No more input is needed from the user.

Simply

In [7]:
from workflow.stage1 import create_ifu_driver_cat

driver_catalog_filename = os.path.join(output_dir, '{}_{}-ifu_driver_cat.fits'.format(survey, trimester))

create_ifu_driver_cat(ifu_driver_template, data_dict, driver_catalog_filename,
                      trimester, PI_email, report_verbosity=report_verbosity,
                      cc_report=cc_report, overwrite=True)

'output/WC_2021A1-ifu_driver_cat.fits'

and check the results:

In [8]:
from workflow.stage1 import check_ifu_driver_cat
from workflow.utils.get_resources import (get_progtemp_file,get_obstemp_file)

progtemp_file=os.path.join('aux', 'progtemp.dat')
obstemp_file=os.path.join('aux', 'obstemp.dat')

if not os.path.exists(progtemp_file):
    logging.info('Downloading the progtemp file')
    get_progtemp_file(file_path=progtemp_file)

if not os.path.exists(obstemp_file):
    logging.info('Downloading the obstemp file')
    get_obstemp_file(file_path=obstemp_file)
    

if check_ifu_driver_cat(driver_catalog_filename, template=ifu_driver_template,
                         progtemp_file=progtemp_file,
                         obstemp_file=obstemp_file):
    print('{} passes the checks'.format(driver_catalog_filename))
else:
    print('{} FAILS the checks'.format(driver_catalog_filename))

output/WC_2021A1-ifu_driver_cat.fits has passed the checks
output/WC_2021A1-ifu_driver_cat.fits passes the checks


Now plot the results:

In [9]:
from workflow.stage1 import plot_ifu_driver_cat

plot_ifu_driver_cat(driver_catalog_filename, output_dir='output',
                        aladin_jar='/Applications/Aladin.app/Contents/Resources/Java/Aladin.jar',
                        progtemp_file=progtemp_file)

This plotting tool needs time. If you want to monitor its progress, see the directory which will contain its ouput images.


A few windows (one per target in `data_dict`) might pop up during the process. Check the images that are saved in the `output` directory.

## Stage 2: Creation of XML files for the targets in the catalog

*Warning*: By default, old XML files will not be overwritten. Instead, new files will be created, continuing the numbering!

This is usually not my intention, so I'll `rm output/*.xml`. Please skip or comment the following line if you would like to *append* the current driver catalogue to the existing target list.

In [10]:
xml_filename = os.path.join(output_dir, '{}_{}-*.xml'.format(survey, trimester))
!rm $xml_filename

rm: No match.


In [11]:
from workflow.stage2 import create_xml_files
from workflow.utils.get_resources import get_blank_xml_template

xml_template='aux/BlankXMLTemplate.xml'

if not os.path.exists(xml_template):
        logging.info('Downloading the blank XML template')
        get_blank_xml_template(file_path=xml_template)

create_xml_files(driver_catalog_filename, output_dir, xml_template,
                     progtemp_file=progtemp_file,
                     obstemp_file=obstemp_file,
                     overwrite=True)


Reading output/WC_2021A1-ifu_driver_cat.fits
Processing 3 non-custom dither OBs for LIFU
	Writing to output/WC_2021A1-lifu_01-t.xml
	Writing to output/WC_2021A1-lifu_02-t.xml
	Writing to output/WC_2021A1-lifu_03-t.xml
Processing 0 custom dither OBs for LIFU
Processing 0 mIFU fields into 0 OBs according to all mode


['output/WC_2021A1-lifu_01-t.xml',
 'output/WC_2021A1-lifu_02-t.xml',
 'output/WC_2021A1-lifu_03-t.xml']

Note two issues:
- the \<field\> element (currently) needs to have the RA_d and DEC_d fields input by hand, due to a bug in Configure (v3.0.2). This will be fixed in Configure ASAP.
- in this case (PROGTEMP='41332'), the binning_Y field is wrong in the XML -- it should be "2", not "1". This also should be fixed (currently by hand) -- Luis is on it!

## Stage 3: Adding guide and calibration stars to the XML files

In [12]:
xml_filename = os.path.join(output_dir, '{}_{}-*-t.xml'.format(survey, trimester))
print(xml_filename)

xml_filename_list = glob.glob(xml_filename)
print(xml_filename_list)
#!export PYTHONPATH='..'; stage3/add_guide_and_calib_stars.py $xml_filename

from workflow.stage3 import add_guide_and_calib_stars

add_guide_and_calib_stars(xml_filename_list, output_dir,
        lifu_num_guide_stars_request=1,
#        num_calib_stars_request=0,
        overwrite=True)

output/WC_2021A1-*-t.xml
['output/WC_2021A1-lifu_02-t.xml', 'output/WC_2021A1-lifu_03-t.xml', 'output/WC_2021A1-lifu_01-t.xml']
Do not trust on the output of this code by now: This software is great, but the assumed position of the LIFU guide camera must be confirmed
825 guide stars have been retrieved for (239.58332953116175, 27.233415952804936)
11 guide stars are useful for (239.58332953116175, 27.233415952804936)
1 guide stars has been selected for (239.58332953116175, 27.233415952804936) PA = 14.399854389500405 deg
Notice that GAIA_DR value for the selected stars is GDR2
Requested value for PA has NOT been adopted
	Writing to output/WC_2021A1-lifu_01-tgc.xml
Do not trust on the output of this code by now: This software is great, but the assumed position of the LIFU guide camera must be confirmed
627 guide stars have been retrieved for (239.7114920447609, 27.393754831677477)
13 guide stars are useful for (239.7114920447609, 27.393754831677477)
1 guide stars has been selected for (23

['output/WC_2021A1-lifu_01-tgc.xml',
 'output/WC_2021A1-lifu_02-tgc.xml',
 'output/WC_2021A1-lifu_03-tgc.xml']

Now check the images `*tgc-guide_stars.png` in the `output` directory, which contain information about the guide stars and the final position angle adopted for the observation, which might not be the exactly the same as the requested PA.

## Stage 4: Configuring the XML files

Now it is time to run the configure scripts. Note that I haven't been able to run these directly from inside the notebook, so I'm dumping the commands below and then running from the command line by hand. (Sorry, I'm not a Jupyter notebook expert!)

In [20]:
configure_script = os.path.join(path_to_configure_tools, 'lifu_configure')
dither_script = os.path.join(path_to_configure_tools, 'dither')

xml_tgc_filename = os.path.join(output_dir, '{}_{}-*-tgc.xml'.format(survey, trimester))
for filename in glob.glob(xml_tgc_filename):
    #print(filename)
    tmp_filename = filename[:-4]+'s-tmp-orig.xml'
    new_filename = filename[:-4]+'s-orig.xml'
    lint_filename = filename[:-4]+'s.xml'
    print('/Users/sctrager/Library/Mobile\ Documents/com~apple~CloudDocs/WEAVE/Configure/configure.app/Contents/MacOS/lifu_configure --epoch 2020 -i {} -o {}'.format(filename,tmp_filename))
#    !/Users/sctrager/Library/Mobile\ Documents/com~apple~CloudDocs/WEAVE/Configure/configure.app/Contents/MacOS/lifu_configure --epoch 2020 -i $filename -o $tmp_filename
    print('/Users/sctrager/Library/Mobile\ Documents/com~apple~CloudDocs/WEAVE/Configure/configure.app/Contents/MacOS/dither -i {} -o {}'.format(tmp_filename,new_filename))
#    !/Users/sctrager/Library/Mobile\ Documents/com~apple~CloudDocs/WEAVE/Configure/configure.app/Contents/MacOS/dither -i $tmp_filename -o $new_filename
    print('xmllint --c14n {} | xmllint --format --encode utf-8 --output {} -'.format(new_filename,lint_filename))
#    !xmllint --c14n $new_filename | xmllint --format --encode utf-8 --output $lint_filename -

/Users/sctrager/Library/Mobile\ Documents/com~apple~CloudDocs/WEAVE/Configure/configure.app/Contents/MacOS/lifu_configure --epoch 2020 -i output/WC_2021A1-lifu_01-tgc.xml -o output/WC_2021A1-lifu_01-tgcs-tmp-orig.xml
/Users/sctrager/Library/Mobile\ Documents/com~apple~CloudDocs/WEAVE/Configure/configure.app/Contents/MacOS/dither -i output/WC_2021A1-lifu_01-tgcs-tmp-orig.xml -o output/WC_2021A1-lifu_01-tgcs-orig.xml
xmllint --c14n output/WC_2021A1-lifu_01-tgcs-orig.xml | xmllint --format --encode utf-8 --output output/WC_2021A1-lifu_01-tgcs.xml -
/Users/sctrager/Library/Mobile\ Documents/com~apple~CloudDocs/WEAVE/Configure/configure.app/Contents/MacOS/lifu_configure --epoch 2020 -i output/WC_2021A1-lifu_03-tgc.xml -o output/WC_2021A1-lifu_03-tgcs-tmp-orig.xml
/Users/sctrager/Library/Mobile\ Documents/com~apple~CloudDocs/WEAVE/Configure/configure.app/Contents/MacOS/dither -i output/WC_2021A1-lifu_03-tgcs-tmp-orig.xml -o output/WC_2021A1-lifu_03-tgcs-orig.xml
xmllint --c14n output/WC_2021

## Stage 5: Creating the IFU FITS catalogue

In [21]:
xml_tgcs_filename = os.path.join(output_dir, '{}_{}-*-tgcs.xml'.format(survey, trimester))
#!export PYTHONPATH='..'; stage5/create_ifu_fits_cat.py $fits_catalogue_template $xml_tgcs_filename --cat_nme1 $PI_first_name --cat_nme2 $PI_surname --gaia_dr 2

xml_tgcs_filename_list=glob.glob(xml_tgcs_filename)

from workflow.stage5 import create_ifu_fits_cat

create_ifu_fits_cat(fits_catalogue_template, xml_tgcs_filename_list,
                        cat_nme1=PI_first_name, cat_nme2=PI_surname,
                        gaia_dr=2, overwrite=True)

'output/WC_2021A1-ifu_from_xmls.fits'

## Stage 6: Improving the IFU FITS catalogue (optional, but recommended)

You can now build integrated-fibre photometry from PanSTARRS-1 using *photometry_from_PS1.py*, which will download and photometer the appropriate PS1 images, or from SDSS using *photometry_from_SDSS.py*, where you first have to download and construct the images (using *SWarp*) via the web interface at [the SDSS DR12 image mosaic download facility](https://dr12.sdss.org/mosaics). If you are using SDSS photometry, make sure that the mosaics that you constructed are in the `output/` directory with names corresponding the object names you used above before running *photometry_from_SDSS.py*.

In [22]:
fits_catalogue_filename = os.path.join(output_dir, '{}_{}-ifu_from_xmls.fits'.format(survey, trimester))
#!export PYTHONPATH='..'; stage6/add_PS1_photometry.py $fits_catalogue_filename
#!setenv PYTHONPATH='${PYTHONPATH}:..'; stage6/add_PS1_photometry.py $fits_catalogue_filename

from workflow.stage6 import photometry_from_PS1

photometry_from_PS1(fits_catalogue_filename)


A2142center: center=(239.58305180295804, 27.233703910368988); size=(0.1574197289063072 deg, 2266 pix)
A2142center_g.fits already exists
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.


this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
../workflow/stage6/photometry_from_PS1.py:171: RuntimeWarning: invalid value encountered in log10
  photo = zero_point - 2.5*np.log10(


A2142center_r.fits already exists
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
trying to change TARGUSE


this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
../workflow/stage6/photometry_from_PS1.py:171: RuntimeWarning: invalid value encountered in log10
  photo = zero_point - 2.5*np.log10(


A2142center_i.fits already exists
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.


...

trying to change TARGUSE


../workflow/stage6/photometry_from_PS1.py:171: RuntimeWarning: invalid value encountered in log10
  photo = zero_point - 2.5*np.log10(


A2142infall_i.fits already exists
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja.


this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
../workflow/stage6/photometry_from_PS1.py:171: RuntimeWarning: invalid value encountered in log10
  photo = zero_point - 2.5*np.log10(


Savingoutput/WC_2021A1-ifu.fits


The broadband images from PanSTARRS are saved in FITS format in a directory with the same name as the original "from_xml" catalog (one file per target per band), as well as PNG previews that illustrate the magnitudes per fibre that go into the final FITS catalogue.
This file is named as the original, stripping the "from_xml" part.

Now it's time for a formal check:

In [23]:
populated_fits_catalogue_filename = os.path.join(output_dir, '{}_{}-ifu.fits'.format(survey, trimester))
#!export PYTHONPATH='..'; stage6/check_populated_ifu_fits_cat.py $fits_catalogue_filename $populated_fits_catalogue_filename

from workflow.stage6 import check_populated_ifu_fits_cat
check_populated_ifu_fits_cat(fits_catalogue_filename, populated_fits_catalogue_filename)


 fitsdiff: 4.0.2
 a: output/WC_2021A1-ifu_from_xmls.fits
 b: output/WC_2021A1-ifu.fits
 HDU(s) not to be compared:
  SIM
 Keyword(s) not to be compared:
  CHECKSUM DATASUM DATETIME MAG_G_CM MAG_I_CM MAG_R_CM
 Table column(s) not to be compared:
  APS_FLAG APS_GAND_EBV APS_GAND_ERR APS_GAND_MODE APS_GAND_RED1
  APS_GAND_RED2 APS_IFU_MASK APS_IFU_SRC_DEC APS_IFU_SRC_ID
  APS_IFU_SRC_RA APS_IFU_TSSL_COVAR APS_IFU_TSSL_MIN_SNR
  APS_IFU_TSSL_TARG_SNR APS_IFU_TSSL_TYPE APS_LS_MODE APS_LS_NUM_MC
  APS_LS_RES APS_PPXF_DEG_ADD APS_PPXF_DEG_MULT APS_PPXF_MOM
  APS_PPXF_NUM_MC APS_PPXF_WL_MAX APS_PPXF_WL_MIN APS_SIGMA
  APS_SSP_NUM_CHAIN APS_SSP_NUM_WLKR APS_TEMPL_LIB APS_TEMPL_LIB_NORM
  APS_WL_MAX APS_WL_MIN APS_Z GAIA_MAG_BP GAIA_MAG_BP_ERR GAIA_MAG_G
  GAIA_MAG_G_ERR GAIA_MAG_RP GAIA_MAG_RP_ERR MAG_G MAG_G_ERR MAG_I
  MAG_I_ERR MAG_R MAG_R_ERR PHOTOZCAT PHOTOZCAT_DEC PHOTOZCAT_DR
  PHOTOZCAT_ID PHOTOZCAT_RA PHOTOZCAT_Z PHOTOZCAT_Z_ERR SDSS_DEC
  SDSS_DR SDSS_FIBRE_MAG_G SDSS_FIBRE_MAG_G_ERR

True

## Stage 7: Creation of the combo FITS catalogue

If you had a mixture of LIFU, mIFU, and MOS observations. We will ignore it for the time being.

In [24]:
# mos_filename = os.path.join(output_dir, '{}_{}-mos.fits'.format(survey, trimester))
final_catalogue_filename = populated_fits_catalogue_filename
# !export PYTHONPATH='..'; stage7/create_combo_fits_cat.py $mos_filename $populated_fits_catalogue_filename

## Stage 8: Submission to WASP of the combo FITS catalogue

This is the time to submit your populated FITS catalogue to the [WASP](http://wasp.ast.cam.ac.uk/) in order to get unique `CNAME` identifiers.

## Stage 9: Finishing the XML files

Now we go from FITS to XML format:

In [25]:
#!export PYTHONPATH='..'; stage9/fill_xmls_with_fits_info.py $final_catalogue_filename $xml_tgcs_filename

from workflow.stage9 import fill_xmls_with_fits_info
fill_xmls_with_fits_info(final_catalogue_filename, xml_tgcs_filename_list, output_dir,
                             overwrite=False)

Skipping file because it already exists: output/WC_2021A1-lifu_01.xml
Skipping file because it already exists: output/WC_2021A1-lifu_02.xml
Skipping file because it already exists: output/WC_2021A1-lifu_03.xml


['output/WC_2021A1-lifu_01.xml',
 'output/WC_2021A1-lifu_02.xml',
 'output/WC_2021A1-lifu_03.xml']

## Stage 10: Submission of the final XML files to WASP

Just a final check:

In [26]:
final_xml_filename = os.path.join(output_dir, '{}_{}-lifu_[0-9][0-9].xml'.format(survey, trimester))
#!export PYTHONPATH='..'; stage10/plot_data_from_xml.py $final_xml_filename

final_xml_filename_list=glob.glob(final_xml_filename)

from workflow.stage10 import plot_data_from_xml

for final_xml_file in final_xml_filename_list:
    plot_data_from_xml(final_xml_file, output_dir=output_dir, verbose=False)

starting plots for file output/WC_2021A1-lifu_01.pdf
	plotting pointings of output/WC_2021A1-lifu_01.xml with color
	plotting pointings of output/WC_2021A1-lifu_01.xml in black and white
	plotting targuse of output/WC_2021A1-lifu_01.xml
	plotting targclass of output/WC_2021A1-lifu_01.xml
	plotting mag_g of output/WC_2021A1-lifu_01.xml
	plotting mag_r of output/WC_2021A1-lifu_01.xml
	plotting mag_i of output/WC_2021A1-lifu_01.xml
	plotting mag_gg of output/WC_2021A1-lifu_01.xml
	plotting mag_bp of output/WC_2021A1-lifu_01.xml
	plotting mag_rp of output/WC_2021A1-lifu_01.xml
starting plots for file output/WC_2021A1-lifu_03.pdf
	plotting pointings of output/WC_2021A1-lifu_03.xml with color
	plotting pointings of output/WC_2021A1-lifu_03.xml in black and white
	plotting targuse of output/WC_2021A1-lifu_03.xml
	plotting targclass of output/WC_2021A1-lifu_03.xml
	plotting mag_g of output/WC_2021A1-lifu_03.xml
	plotting mag_r of output/WC_2021A1-lifu_03.xml
	plotting mag_i of output/WC_2021A1

and, after verifying that everything seems to make sense, the files should be packed together into a single tar file and uploaded to the [WASP](http://wasp.ast.cam.ac.uk/).